In [47]:
import pandas as pd

df = pd.read_csv('../project_assets/fema_disasters_1994_2020.csv')

# Show columns and data types
df.dtypes

femaDeclarationString        object
disasterNumber              float64
state                        object
declarationType              object
declarationDate              object
fyDeclared                  float64
incidentType                 object
declarationTitle             object
ihProgramDeclared           float64
iaProgramDeclared           float64
paProgramDeclared           float64
hmProgramDeclared           float64
Date                         object
incidentBeginDate            object
incidentEndDate              object
disasterCloseoutDate         object
fipsStateCode               float64
fipsCountyCode              float64
placeCode                   float64
designatedArea               object
declarationRequestNumber    float64
hash                         object
lastRefresh                  object
id                           object
dtype: object

In [52]:
# Create a result DataFrame
results = df

# Filters
keep_columns = [
    'femaDeclarationString', 
    'state',
    'designatedArea',
    'fipsCountyCode',
    'declarationType',
    'declarationTitle',
    'incidentType',
    'incidentBeginDate'
]
state_filter = df['state']=='FL'
is_county = df['fipsCountyCode'] > 0

# Transformations (Rename)
renamed_columns = {
    'femaDeclarationString': 'disaster_id',
    'state': 'disaster_state',
    'designatedArea': 'disaster_county',
    'fipsCountyCode': 'disaster_county_code',
    'declarationType': 'disaster_declaration',
    'declarationTitle': 'disaster_name',
    'incidentType': 'disaster_type',
    'incidentBeginDate': 'disaster_date'
}

# Apply filters and transformations
results = df[state_filter & is_county][keep_columns]
results.rename(columns=renamed_columns, inplace=True)

# Change Data Types
results = results.astype(str)

results.nunique()

disaster_id             125
disaster_state            1
disaster_county          68
disaster_county_code     67
disaster_declaration      3
disaster_name            98
disaster_type             8
disaster_date           107
dtype: int64

In [49]:
# Export to CSV
results.to_csv('fema_fl_disasters_1994_2020.csv', index=False)

In [46]:
# Testing Outputs
group_by = ['disaster_county', 'disaster_county_code']
results.groupby(group_by)['disaster_id'].nunique().tail(10)

disaster_county      disaster_county_code
St. Johns (County)   109.0                   21
St. Lucie (County)   111.0                   29
Sumter (County)      119.0                   18
Suwannee (County)    121.0                   24
Taylor (County)      123.0                   23
Union (County)       125.0                   20
Volusia (County)     127.0                   31
Wakulla (County)     129.0                   30
Walton (County)      131.0                   30
Washington (County)  133.0                   28
Name: disaster_id, dtype: int64